In [1]:
import os
import re
from CameraDocument import CameraDocument
from glob import glob

def extract_number(file_path):
    match = re.search(r'(\d+)\.json$', file_path)
    return int(match.group(1)) if match else None

camera_model_list = ["gfx100ii", "x-e4", "x-s20", "x-t5", 'x100v']
json_dir = "../indexing/data/json"

documents = []
for camera_model in camera_model_list:
    detail_json_dir = os.path.join(json_dir, camera_model, "LlamaParseMultimodal", "processed_data")
    json_path = os.path.join(detail_json_dir, "*.json")
    json_list = glob(json_path)
    json_list = sorted(json_list, key=extract_number)

    for i, path in enumerate(json_list):
        document = CameraDocument()
        document.load_json(path)
        documents.append(document)

In [4]:
import pandas as pd

doc_id = []
contents = []
metadata = []
for i, document in enumerate(documents):
    doc_id.append(str(i))
    contents.append(document.parsing_result)
    metadata.append(document.metadata)
    
corpus_df = pd.DataFrame(
    {
        "doc_id": doc_id,
        "contents": contents,
        "metadata": metadata
    }
)

In [6]:
corpus_df

,doc_id,contents,metadata
0,0,# Parts of the Camera\n\n!Parts of the Camera\...,"{'page': 2, 'model': 'gfx100ii', 'chapter': 'B..."
1,1,# Parts of the Camera\n\n!Camera Diagram\n\n##...,"{'page': 3, 'model': 'gfx100ii', 'chapter': 'B..."
2,2,# Before You Begin\n\n!Camera Diagram\n\n### D...,"{'page': 4, 'model': 'gfx100ii', 'chapter': 'B..."
3,3,# The Cable Protector\n\nAttach the protector ...,"{'page': 5, 'model': 'gfx100ii', 'chapter': 'B..."
4,4,# Before You Begin\n\n## The Serial Number Pla...,"{'page': 6, 'model': 'gfx100ii', 'chapter': 'B..."
...,...,...,...
1838,1838,# Specifications\n\n## System\n\n### Metering\...,"{'page': 319, 'model': 'x100v', 'subsection': ..."
1839,1839,\n# System\n\n## Continuous\n\n| CONTINUOUS MO...,"{'page': 320, 'model': 'x100v', 'subsection': ..."
1840,1840,\n# Specifications\n\n## System\n\n### Flash M...,"{'page': 321, 'model': 'x100v', 'subsection': ..."
1841,1841,\n# Power Supply/Other\n\n## Power Supply\n- *...,"{'page': 322, 'model': 'x100v', 'subsection': ..."


In [5]:
corpus_df.to_parquet("0.parquet")

In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)

True

In [2]:
from autorag.data.qa.schema import Raw, Corpus
import pandas as pd

raw_df = pd.read_parquet("0.parquet") # The folder path must be changed up to v0.3.8
raw_instance = Raw(raw_df)

corpus_df = pd.read_parquet("0.parquet") # The folder path must be changed up to v0.3.8
corpus_instance = Corpus(corpus_df, raw_instance)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


[01/31/25 17:10:05] INFO     [__init__.py:81] >> You are using API version of AutoRAG.To use local   ]8;id=256937;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/autorag/__init__.py\__init__.py]8;;\:]8;id=98834;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/autorag/__init__.py#81\81]8;;\
                             version, run pip install 'AutoRAG[gpu]'                                               

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
import pandas as pd
from llama_index.llms.openai import OpenAI

from autorag.data.qa.filter.dontknow import dontknow_filter_rule_based
from autorag.data.qa.generation_gt.llama_index_gen_gt import (
    make_concise_gen_gt,
)
from autorag.data.qa.query.llama_gen_query import factoid_query_gen
from autorag.data.qa.sample import random_single_hop

llm = OpenAI(model="gpt-4o")

initial_qa = (
    corpus_instance.sample(random_single_hop, n=100) # Only sample three passages. This will make three QA dataset only. Needs to adjust before real use.
    .map(
        lambda df: df.reset_index(drop=True), # Drop index
    )
    .make_retrieval_gt_contents() # This is essential for making questions with gt (ground truth) passages.
    .batch_apply(
        factoid_query_gen,  # query generation
        llm=llm,
    )
    .batch_apply(
        make_concise_gen_gt,  # answer generation (concise)
        llm=llm,
    )
    .filter(
        dontknow_filter_rule_based,  # unanswerable filter
        lang="en",
    )
)

initial_qa.to_parquet('initial_qa.parquet', 'initial_corpus.parquet')

[01/31/25 17:10:10] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=438672;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=502340;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=191870;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=784346;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/31/25 17:10:11] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=773076;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=836977;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=39950;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=595722;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=268780;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=282953;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=446847;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=371020;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/31/25 17:11:09] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=943543;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=468822;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=412103;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=372166;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/31/25 17:11:12] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=359157;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=150638;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=626170;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=677509;file:///home/jeongho/RAGprojects/autorag_env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [5]:
pd.set_option('display.max_rows', None)      # 모든 행 출력
pd.set_option('display.max_columns', None)   # 모든 열 출력
pd.set_option('display.max_colwidth', None)  # 문자열 길이 제한 없음
pd.set_option('display.expand_frame_repr', False)  # 가로로 긴 데이터도 생략 없이 출력


In [6]:
pd.read_parquet("initial_qa.parquet")

,qid,query,retrieval_gt,generation_gt
0,840fc0d2-bc18-4152-ab0d-4f064450c138,How do you enable and use the Movie Silent Control feature to adjust movie settings without recording camera control sounds?,[[1559]],"[To enable and use the **Movie Silent Control** feature to adjust movie settings without recording camera control sounds, follow these steps:\n\n1. **Enable Movie Silent Control**:\n - Go to the **SHOOTING MENU** on your camera.\n - Select **MOVIE SETTING > MOVIE SILENT CONTROL**.\n - Choose **ON** to activate the feature.\n\n2. **Access Silent Controls**:\n - Once enabled, a **!SET icon** will appear on the LCD monitor.\n - Tap the **!SET icon** to access the touch-screen controls.\n\n3. **Adjust Movie Settings**:\n - Use the touch-screen to adjust the following settings:\n - **SHUTTER SPEED**\n - **APERTURE**\n - **EXPOSURE COMPENSATION**\n - **ISO**\n - **INTERNAL MIC LEVEL ADJUSTMENT/EXTERNAL MIC LEVEL ADJUSTMENT**\n - **WIND FILTER**\n - **HEADPHONES VOLUME**\n - **FILM SIMULATION**\n - **WHITE BALANCE**\n\n**Note**: Enabling **Movie Silent Control** will disable the physical dial controls on the camera, allowing for silent adjustments via the touch-screen.]"
1,0fa36f45-d8b4-4023-aad8-08101e65b70c,How should you store and care for the camera to ensure its longevity when it is not in use for an extended period?,[[1501]],"[To store and care for the camera to ensure its longevity when it is not in use for an extended period, follow these steps:\n\n1. **Remove the Battery and Memory Card**: \n - Ensure that both the battery and memory card are removed from the camera to prevent any potential damage or data loss.\n\n2. **Avoid Harmful Environments**: \n - Do not store or use the camera in locations that are:\n - **Exposed to rain, steam, or smoke**.\n - **Very humid or extremely dusty**.\n - **Exposed to direct sunlight or very high temperatures**, such as in a closed vehicle on a sunny day.\n - **Extremely cold**.\n - **Subject to strong vibration**.\n - **Exposed to strong magnetic fields**, such as near a broadcasting antenna, power line, radar emitter, motor, transformer, or magnet.\n - **In contact with volatile chemicals** such as pesticides.\n - **Next to rubber or vinyl products**.\n\nBy following these precautions, you can help ensure the camera remains in good condition during periods of non-use.]"
2,b44b1674-7507-42fb-8aab-10e63aaaebaa,How do you enable or disable the touch-screen controls for shooting on the camera?,[[352]],"[To enable or disable the touch-screen controls for shooting on the camera, follow these steps:\n\n1. **Access the Touch Screen Setting**: Navigate to the **Touch Screen Setting** menu on your camera.\n2. **Select the Desired Option**:\n - Choose **ON** to enable the LCD monitor to function as a touch screen during shooting.\n - Choose **OFF** to disable touch controls.\n\nBy following these steps, you can control whether the touch-screen functionality is active during shooting.]"
3,44d2a684-62d3-4246-a50f-eb47545beac0,How do you save changes to the image quality settings and confirm the selection using the camera's menu options?,[[1647]],"[To save changes to the image quality settings and confirm the selection using the camera's menu options, follow these steps:\n\n1. **Save Changes**: \n - After adjusting the desired image quality settings, press **MENU/OK** to save changes to the selected item and return to the menu item list.\n\n2. **Confirm Selection**: \n - Press **DISP/BACK** while in the menu item list to display a confirmation dialog.\n - Highlight **OK** and press **MENU/OK** to save the settings to the selected bank.\n\nBy following these steps, you can effectively save and confirm your image quality settings on the camera.]"
4,810f3817-3c69-451e-928b-752efa1bf271,How do you customize the roles of the lens function buttons on the camera to suit your shooting preferences?,[[1463]],"[To customize the roles of the lens function buttons on the camera, follow these steps:\n\n1. **

In [9]:
from autorag.evaluator import Evaluator

evaluator = Evaluator(qa_data_path='initial_qa.parquet', corpus_data_path='initial_corpus.parquet', project_dir='project_dir3')
evaluator.start_trial('full.yaml', skip_validation = True)

ModuleNotFoundError: No module named 'torch'